In [2]:
from tensorflow import keras
import pandas as pd
import numpy as np
from joblib import load

# loading the model
model = keras.models.load_model("/content/drive/MyDrive/machine_learning/models/finstreet_new.keras")

# loading the data scaler
scaler = load("/content/drive/MyDrive/machine_learning/models/min_max_scaler.joblib")

# loading the csv file
data = pd.read_csv("/content/drive/MyDrive/machine_learning/SONATA.csv")
data = data[["Open", "High", "Low", "Close"]]
scaled_data = scaler.transform(data) # Convert data to 0-1 range


In [13]:
from datetime import datetime, timedelta
# Since the model was trained to predict hourly data
# new predictions will be houly from 2025-12-30 15:15:00+05:30
# The window size if of 12H
window_size = 12
last_window = scaled_data[len(scaled_data)-window_size:len(scaled_data)]

start_time = datetime(2025, 12, 30, 15, 15, 0)
cur_time = start_time

predicted_data = []

x_new = last_window.reshape(1, last_window.shape[0], last_window.shape[1])
while (cur_time <= datetime(2026, 1, 8)):

  # Track of the prediction time
  if (cur_time.hour < 15):
    cur_time = cur_time+timedelta(hours=1)
  else:
    cur_time = cur_time+timedelta(days=1)
    cur_time = cur_time.replace(hour=9)

  y_new = model.predict(x_new)
  x_new = np.concatenate((x_new[:, 1:, :], y_new.reshape(1, y_new.shape[0], y_new.shape[1])), axis=1)

  y_pred = scaler.inverse_transform(y_new)
  pred_new = [cur_time.isoformat()]
  pred_new.extend(y_pred[0])
  predicted_data.append(pred_new)

predicted_df = pd.DataFrame(predicted_data, columns=["Datetime", "Open", "High", "Low", "Close"])
predicted_df.head()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━

,Datetime,Open,High,Low,Close
0,2025-12-31T09:15:00,358.719666,360.398499,357.096558,358.479370
1,2025-12-31T10:15:00,359.240173,360.931458,357.561737,358.946625
2,2025-12-31T11:15:00,359.659821,361.349945,357.947174,359.326691
3,2025-12-31T12:15:00,360.030090,361.716675,358.293671,359.667816
4,2025-12-31T13:15:00,360.366425,362.053802,358.604889,359.985291


In [12]:
predicted_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Datetime  57 non-null     object 
 1   Open      57 non-null     float32
 2   High      57 non-null     float32
 3   Low       57 non-null     float32
 4   Close     57 non-null     float32
dtypes: float32(4), object(1)
memory usage: 1.5+ KB


In [14]:
predicted_df.to_csv("/content/drive/MyDrive/machine_learning/predictions.csv")